In [ ]:
import subprocess
from datetime import datetime
from pathlib import Path

import fsspec
from utils import sync_notebook

In [ ]:
fs = fsspec.filesystem("file")
current_date = datetime.today().strftime("%Y-%m-%d")
output_folder = f"results/{current_date}/"
fs.mkdirs(output_folder, exist_ok=True)

In [ ]:
dataset = "gpm_imerg"

In [ ]:
def run_memray(file: str, zoom: str):
    output_file = f"{output_folder}memray-{dataset}-{Path(file).stem}-{zoom}.bin"
    summary_file = f"{output_folder}memray-{dataset}-{Path(file).stem}-{zoom}.json"
    command = [
        "memray",
        "run",
        "--force",
        "--output",
        output_file,
        file,
        "--dataset",
        dataset,
        "--zoom",
        zoom,
    ]
    subprocess.run(command)
    summary_command = [
        "memray",
        "stats",
        "--force",
        "--json",
        "--output",
        summary_file,
        output_file,
    ]
    subprocess.run(summary_command)

In [ ]:
def sync_and_run(zoom, input_methods):
    notebooks = []
    for fp in input_methods:
        notebooks.extend(fs.glob(f"resample-*-{fp}*.ipynb"))
    for file in notebooks:
        sync_notebook(file)
    modules = []
    for fp in input_methods:
        modules.extend(fs.glob(f"resample-*-{fp}*.py"))
    for file in modules:
        # Skip kerchunk since it requires a different image build due to incompatibility with Zarr V3
        if "kerchunk" not in file:
            run_memray(file, zoom)

In [ ]:
if dataset == "gpm_imerg":
    input_methods = ["rioxarray", "odc", "pyresample", "rioxarray", "xesmf", "sparse"]
    zoom_levels = ["0", "1", "2"]
elif dataset == "mursst":
    input_methods = ["odc", "pyresample", "rioxarray", "rasterio"]
    zoom_levels = ["0", "1", "2", "3", "4", "5", "6"]

In [ ]:
for zoom in zoom_levels:
    sync_and_run(zoom, input_methods)